In [138]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold
from PIL import Image
from torchmetrics import R2Score

# Define custom dataset
class PlantDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_id = str(self.data_frame.iloc[idx, 0])
        img_path = os.path.join(self.root_dir, img_id + '.jpeg')
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        ancillary_data = self.data_frame.iloc[idx, 1:164].values.astype('float32')
        if 'train' in self.root_dir:
            labels = self.data_frame.iloc[idx, 164:].values.astype('float32')
            return [ancillary_data, image], labels
        else:
            return [ancillary_data, image], img_id

# Define transformations
data_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    # No noise needed since clear images
])

# Load the datasets
train_dataset = PlantDataset(csv_file='data/train_preprocessed.csv', root_dir='data/train_images', transform=data_transforms)
test_dataset = PlantDataset(csv_file='data/test_preprocessed.csv', root_dir='data/test_images', transform=data_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, pin_memory=True)


In [139]:
print(len(train_dataset))
print(len(test_dataset))
print(len(train_loader))
print(len(test_loader))

43363
6391
1356
200


In [140]:
data_frame = pd.read_csv("data/train.csv")
# log_columns = data_frame.columns[7:]
# data_frame[log_columns] = data_frame[log_columns].apply(np.log10)
# data_frame[log_columns] = data_frame[log_columns].replace(-np.inf, 0)
# data_frame[log_columns] = data_frame[log_columns].fillna(0)

target_columns = data_frame.columns[164:]
infer_min = data_frame[target_columns].min()
infer_max = data_frame[target_columns].max()
print(f"Min values: {infer_min.values}")
print(f"Max values: {infer_max.values}")

Min values: [2.92916029e-01 1.33873997e+02 1.96990260e+04 3.45940007e+03
 1.36050911e+01 3.97556994e+05]
Max values: [1.37534090e+00 1.74258658e+02 1.97225418e+04 4.17983799e+03
 1.71674261e+01 4.13183477e+05]


In [141]:
class ResNet18_WithAncillary_Linear(nn.Module):
    def __init__(self):
        super(ResNet18_WithAncillary_Linear, self).__init__()
        self.cnn = models.resnet18(weights='DEFAULT')
        self.fc = nn.Linear(self.cnn.fc.in_features + 163, 6)  # Logistic regression layer
        self.cnn.fc = nn.Identity()  # Remove the last fully connected layer

    def forward(self, x):
        ancillary_data, images = x
        cnn_out = self.cnn(images)
        combined = torch.cat((cnn_out, ancillary_data), dim=1)
        out = self.fc(combined)
        return out

In [142]:
class Ensemble_ResNet18(nn.Module):
    def __init__(self):
        super(Ensemble_ResNet18, self).__init__()
        self.base_cnn = models.resnet18(weights='DEFAULT')
        self.base_cnn.fc = nn.Identity()

        self.cnn_output = nn.Sequential(
            nn.Linear(512, 64),
            nn.ReLU(),
            nn.Linear(64, 4)
        )

        self.output_dense = nn.Sequential(
            nn.Linear(163, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 4)
        )

        self.main_output = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        ancillary_data, images = x
        output_cnn = self.cnn_output(self.base_cnn(images))
        output_dense = self.output_dense(ancillary_data)

        combined = torch.cat((output_cnn, output_dense), dim=1)
        return self.main_output(combined)

In [143]:
class Ensemble_ResNet50(nn.Module):
    def __init__(self):
        super(Ensemble_ResNet50, self).__init__()
        self.base_cnn = models.resnet50(weights='DEFAULT')
        self.base_cnn.fc = nn.Identity()

        self.cnn_output = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 4)
        )

        self.output_dense = nn.Sequential(
            nn.Linear(163, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 4)
        )

        self.main_output = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        ancillary_data, images = x
        output_cnn = self.cnn_output(self.base_cnn(images))
        output_dense = self.output_dense(ancillary_data)

        combined = torch.cat((output_cnn, output_dense), dim=1)
        return self.main_output(combined)

In [144]:
class Ensemble_ResNet101(nn.Module):
    def __init__(self):
        super(Ensemble_ResNet101, self).__init__()
        self.base_cnn = models.resnet101(weights='DEFAULT')
        self.base_cnn.fc = nn.Identity()

        self.cnn_output = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, 4)
        )

        self.output_dense = nn.Sequential(
            nn.Linear(163, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 4)
        )

        self.main_output = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            nn.ReLU(),
            nn.Linear(4, 1)
        )

    def forward(self, x):
        ancillary_data, images = x
        output_cnn = self.cnn_output(self.base_cnn(images))
        output_dense = self.output_dense(ancillary_data)

        combined = torch.cat((output_cnn, output_dense), dim=1)
        return self.main_output(combined)

In [145]:
class ResNettrail(nn.Module):
    def __init__(self):
        super(ResNettrail, self).__init__()
        self.base_cnn = models.resnet50(weights='DEFAULT')
        self.base_cnn.fc = nn.Identity()

        # Unfreeze last few layers in base_cnn
        for param in self.base_cnn.parameters():
            param.requires_grad = False
        for param in self.base_cnn.layer4.parameters():
            param.requires_grad = True

        self.cnn_output = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 32)
        )

        self.output_dense = nn.Sequential(
            nn.Linear(163, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32)
        )

        self.main_output = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            # nn.Linear(64, 64),
            # nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 6)
        )

    def forward(self, x):
        ancillary_data, images = x
        output_cnn = self.cnn_output(self.base_cnn(images))
        output_dense = self.output_dense(ancillary_data)

        combined = torch.cat((output_cnn, output_dense), dim=1)
        return self.main_output(combined)


In [146]:
class ResNettrail2(nn.Module):
    def __init__(self):
        super(ResNettrail2, self).__init__()
        self.base_cnn = models.resnet50(weights='DEFAULT')
        self.base_cnn.fc = nn.Identity()

        # # Unfreeze last few layers in base_cnn
        # for param in self.base_cnn.parameters():
        #     param.requires_grad = False
        # for param in self.base_cnn.layer4.parameters():
        #     param.requires_grad = True

        self.cnn_output = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 32)
        )

        self.output_dense = nn.Sequential(
            nn.Linear(163, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32)
        )

        self.main_output = nn.Sequential(
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 6)
        )

    def forward(self, x):
        ancillary_data, images = x
        output_cnn = self.cnn_output(self.base_cnn(images))
        output_dense = self.output_dense(ancillary_data)

        combined = torch.cat((output_cnn, output_dense), dim=1)
        return self.main_output(combined)


In [147]:
class Encoder_DenseNet201(nn.Module):
    def __init__(self):
        super(Encoder_DenseNet201, self).__init__()
        self.base_cnn = models.densenet201(weights='DEFAULT')

        for param in self.base_cnn.parameters():
            param.requires_grad = False

        self.cnn_output = nn.Sequential(
            nn.Linear(1000, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 32)
        )

        self.output_dense = nn.Sequential(
            nn.Linear(163, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 32)
        )

        self.main_output = nn.Sequential(
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 6)
        )

    def forward(self, x):
        ancillary_data, images = x
        output_cnn = self.cnn_output(self.base_cnn(images))
        output_dense = self.output_dense(ancillary_data)

        combined = torch.cat((output_cnn, output_dense), dim=1)
        return self.main_output(combined)


In [148]:
class ResNettrail_combined(nn.Module):
    def __init__(self):
        super(ResNettrail_combined, self).__init__()
        self.model1 = ResNettrail()
        self.model2 = Encoder_DenseNet201()
        self.model1.load_state_dict(torch.load('best_models/stage4_freeze_cnn.pth'))
        self.model2.load_state_dict(torch.load('best_models2/best_model_1.pth'))

        for param in self.model1.parameters():
            param.requires_grad = False
        for param in self.model2.parameters():
            param.requires_grad = False

    def forward(self, x):
        x1 = self.model1(x)
        x2 = self.model2(x)
        return 1 / 2 * (x1 + x2)

In [149]:
class feature_based_prediction_architech(nn.Module):
    def __init__(self):
        super(feature_based_prediction_architech, self).__init__()
        self.feature0 = Ensemble_ResNet50()
        self.feature0.load_state_dict(torch.load('temp/1_epoch_26_resnet50.pth'))

        self.feature1 = Ensemble_ResNet50()
        self.feature1.load_state_dict(torch.load('temp/2_epoch_25_resnet50.pth'))

        self.feature2 = Ensemble_ResNet18()
        self.feature2.load_state_dict(torch.load('temp/3_epoch_13_resnet18.pth'))

        self.feature3 = Ensemble_ResNet18()
        self.feature3.load_state_dict(torch.load('temp/4_epoch_16_resnet18.pth'))

        self.feature4 = Ensemble_ResNet50()
        self.feature4.load_state_dict(torch.load('temp/5_epoch_16_resnet50.pth'))

        self.feature5 = Ensemble_ResNet50()
        self.feature5.load_state_dict(torch.load('temp/6_epoch_16_resnet50.pth'))

    def forward(self, x):
        x0 = self.feature0(x)
        x1 = self.feature1(x)
        x2 = self.feature2(x)
        x3 = self.feature3(x)
        x4 = self.feature4(x)
        x5 = self.feature5(x)
        return torch.cat((x0, x1, x2, x3, x4, x5), dim=1)

In [158]:
# Instantiate the models, loss function, and optimizers
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [159]:
model = ResNettrail().to(device) # Initialize the model
model.load_state_dict(torch.load('best_models/stage4_freeze_cnn.pth'))  # Load state dictionary

In [153]:
model.eval()
r2_score_metric = R2Score(num_outputs=6).to(device)
train_predictions = []
train_true_values = []

# Generate predictions for the training set
with torch.no_grad():
    for [ancillary_data, images], targets in train_loader:
        ancillary_data, images, targets = ancillary_data.to(device), images.to(device), targets.to(device)
        outputs = model([ancillary_data, images])

        train_predictions.append(outputs)
        train_true_values.append(targets)


# Compute the R² score
train_predictions = torch.cat(train_predictions)
train_true_values = torch.cat(train_true_values)
r2 = r2_score_metric(train_predictions, train_true_values).item()
print(f'R² score on the training set: {r2:.4f}')

c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature 

R² score on the training set: -1291084890112.0000


c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(


In [154]:
# Prediction and Submission
infer_max = torch.tensor(infer_max.to_numpy()).to(device)
infer_min = torch.tensor(infer_min.to_numpy()).to(device)

model.eval()
predictions = []
image_ids = []
with torch.no_grad():
    for [ancillary_data, images], ids in test_loader:
        ancillary_data, images = ancillary_data.to(device), images.to(device)
        outputs = model([ancillary_data, images])
        outputs = outputs * (infer_max - infer_min) + infer_min
        # outputs = 10 ** outputs
        predictions.extend(outputs.cpu().numpy())
        image_ids.extend([int(id) for id in ids])

# Prepare the submission file
submission = pd.DataFrame(predictions, columns=['X4', 'X11', 'X18', 'X26', 'X50', 'X3112'])
submission['id'] = image_ids
submission = submission[['id', 'X4', 'X11', 'X18', 'X26', 'X50', 'X3112']]
submission.to_csv('j228ye_rest18linear.csv', index=False)

c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
c:\Users\YZM\Desktop\for fun\pytorch\torch-playground\lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature 